In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime

# set the graphs to show in the jupyter notebook
%matplotlib inline

# set seaborn graphs to a better style
sns.set(style="ticks")

### Dataset
Utilize os dados do arquivo em <a href="https://archive.ics.uci.edu/ml/datasets/online+retail">'data/Online_Retail.csv'</a> para esses exercícios.
Obs: se encontrar um erro de utf-8 decode, adicione o `encoding = 'latin1'` na leitura dos dados.

### Descrição dos dados


- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction.<br> - **If this code starts with letter 'c', it indicates a cancellation**.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.



In [8]:
df = pd.read_csv('Online_Retail.csv', sep = ',')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2010 08:26,3.39,17850.0,United Kingdom


### Análise geral dos dados
Aproveite esse espaço para entender melhor seus dados, sua estrutura e estatística básica. <br> 
Você consegue perceber algo de estranho?

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [10]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611121,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [11]:
df.isna().sum() / df.shape[0]*100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [12]:
df.duplicated().sum()

5268

#### Problemas encontrados:
- Preços nulos
- Quantidades nulas
- Dados duplicados
- Data/Hora  está como string (não é necessárimente um problema, mas dificulta na hora de trabalhar com a coluna InvoiceDate)

Resolva o problema encontrado nos dados

In [13]:
# removendo o dados duplicados
df.drop_duplicates(keep='first', inplace=True)
df.shape

(536641, 8)

In [14]:
# Removendo os valores de UnitPrice e Quantity menores que 0
novoDF = df.query('UnitPrice > 0')
novoDF = novoDF.query('Quantity > 0 ')
novoDF.describe()

,Quantity,UnitPrice,CustomerID
count,524878.000000,524878.000000,392692.000000
mean,10.616600,3.922580,15287.843865
std,156.280031,36.093027,1713.539549
min,1.000000,0.040000,12346.000000
25%,1.000000,1.250000,13955.000000
50%,4.000000,2.080000,15150.000000
75%,11.000000,4.130000,16791.000000
max,80995.000000,13541.330000,18287.000000


In [15]:
#string_date = "1/12/2010 08:26"
#format = "%d/%m/%Y %H:%M"
#datetime_object = datetime.strptime(string_date, format)

#fomação da data
#df['InvoiceDate'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y %H:%M"))

# foma simples de fazer a conversão
novoDF['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [16]:
novoDF.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [18]:
novoDF.to_csv('dados_editados.csv', index=False) 

In [19]:
# verificando a quantidades de dados nulos que restaram
novoDF.isna().sum() / novoDF.shape[0]*100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.000000
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     25.184138
Country         0.000000
dtype: float64

Como os dados nulos em CustomerID represema 25% dos dados, será feito o preenchimento desses dados, como forma de preservar os dadaos e não haver problemas em futuras análises 


In [25]:
#Máximo valor encontrado para a coluna CustomerID
max(novoDF['CustomerID'].unique())

18287.0

In [21]:
#Criando um novo dataframe com as linhas nulas do CustomerID

nulos_df = novoDF[novoDF['CustomerID'].isna()]
nulos_df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-01-12 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-01-12 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-01-12 14:32:00,0.85,NaN,United Kingdom


In [22]:
nulos_df['CustomerID'].unique()

array([nan])

In [23]:
nulos_df['InvoiceNo'].unique()

array(['536544', '536555', '536558', ..., '581492', '581497', '581498'],
      dtype=object)

In [24]:
len(nulos_df['InvoiceNo'].unique())

1428

In [28]:
nulos_df['CustomerID'].fillna(value = 99999, inplace =True)
nulos_df

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-01-12 14:32:00,2.51,99999.0,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-01-12 14:32:00,2.51,99999.0,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-01-12 14:32:00,0.85,99999.0,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-01-12 14:32:00,1.66,99999.0,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-01-12 14:32:00,1.66,99999.0,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-09-12 10:26:00,4.13,99999.0,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-09-12 10:26:00,4.13,99999.0,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-09-12 10:26:00,4.96,99999.0,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-09-12 10:26:00,10.79,99999.0,United Kingdom


In [29]:
#pegando o dataframe novo apenas com os valores sem nan na coluna CustomerID
fill_df = novoDF[novoDF['CustomerID'].notna()]
fill_df['CustomerID'].isna().sum()

0

In [31]:
#vendo quantas linhas tem o novoDF
novoDF.shape

(524878, 8)

In [32]:
#Concatenando os dois dataframes fill_df e nulos_df em um novo final_df

final_df = pd.concat([fill_df, nulos_df])

final_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom


In [33]:
final_df.shape
#mesmo número de linha do novoDF, com os valores de CustomerID todo preenchido, sendo que os valores iguais a 99999, se refere aqueles registros com dados 
#nulos para esta coluna

(524878, 8)